In [1]:
import torch
import json
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Create Torch Dataset

In [4]:
class keypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.485, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(item['kps']).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps


In [5]:
train_dataset = keypointsDataset("data/images", "data/data_train.json")
val_dataset = keypointsDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

Create Model

In [6]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer

c:\Users\Clayton Tedesco\Desktop\Dev\Tennis Court Analysis\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Clayton Tedesco\Desktop\Dev\Tennis Court Analysis\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Clayton Tedesco/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 25.3MB/s]


In [7]:
model = model.to(device)

Train Model

In [8]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [10]:
num_epochs=20
for epoch in range(num_epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Step {i+1}/{len(train_loader)}, Loss: {loss.item()}")

Epoch 1/20, Step 1/829, Loss: 8.881326675415039
Epoch 1/20, Step 11/829, Loss: 8.392955780029297
Epoch 1/20, Step 21/829, Loss: 23.6874942779541
Epoch 1/20, Step 31/829, Loss: 3.941317319869995
Epoch 1/20, Step 41/829, Loss: 8.781803131103516
Epoch 1/20, Step 51/829, Loss: 8.761395454406738
Epoch 1/20, Step 61/829, Loss: 5.470914363861084
Epoch 1/20, Step 71/829, Loss: 7.358462333679199
Epoch 1/20, Step 81/829, Loss: 7.2312726974487305
Epoch 1/20, Step 91/829, Loss: 10.274041175842285
Epoch 1/20, Step 101/829, Loss: 3.2631380558013916
Epoch 1/20, Step 111/829, Loss: 9.501991271972656
Epoch 1/20, Step 121/829, Loss: 7.398227214813232
Epoch 1/20, Step 131/829, Loss: 6.05501651763916
Epoch 1/20, Step 141/829, Loss: 3.6134145259857178
Epoch 1/20, Step 151/829, Loss: 9.99267578125
Epoch 1/20, Step 161/829, Loss: 13.197402000427246
Epoch 1/20, Step 171/829, Loss: 2.142960786819458
Epoch 1/20, Step 181/829, Loss: 3.523024320602417
Epoch 1/20, Step 191/829, Loss: 12.742523193359375
Epoch 1/20,

In [12]:
torch.save(model.state_dict(), "keypoints_model.pth")